In [74]:
import pandas as pd
import numpy as np
import math

In [ ]:
data = pd.read_csv("lyrics.csv")

In [75]:
data = data.replace({'\[instrumental\]' : math.nan})
data = data.dropna()
data = data[data.genre != "Not Available"]
data = data.replace({'\n': ' '}, regex=True)

In [76]:
len(data)

242615

In [77]:
data.head(10)

,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing? You know I'm gonna cut..."
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy, it's like you seem..."
2,2,honesty,2009,beyonce-knowles,Pop,If you search For tenderness It isn't hard to ...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I [Verse 1:] If I wrote a..."
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."
5,5,all-i-could-do-was-cry,2009,beyonce-knowles,Pop,I heard Church bells ringing I heard A choir s...
6,6,once-in-a-lifetime,2009,beyonce-knowles,Pop,This is just another day that I would spend Wa...
7,7,waiting,2009,beyonce-knowles,Pop,"Waiting, waiting, waiting, waiting Waiting, wa..."
8,8,slow-love,2009,beyonce-knowles,Pop,[Verse 1:] I read all of the magazines while w...
9,9,why-don-t-you-love-me,2009,beyonce-knowles,Pop,"N-n-now, honey You better sit down and look ar..."


In [78]:
training_50k = data.sample(n=50000, replace=True)
validation_50k = data.sample(n=50000, replace=True)
testing_50k = data.sample(n=50000, replace=True)

In [79]:
training_50k.head(10)

,index,song,year,artist,genre,lyrics
349226,349226,cages-explicit,2016,art-of-dying,Rock,IN THIS BODY THERE'S A HEART A HEART THAT'S BE...
335640,335640,patterns,2009,band-of-skulls,Rock,There's one for the sorrow and two for the joy...
75908,75908,ram-jam-city,2006,danny-kirwan,Rock,Written by danny kirwan. Oh my girl you were d...
58501,58501,framed,2011,cirrha-niva,Rock,Lifeless in a matching frame with a smile for ...
227357,227357,here-i-go-again-instrumental,2014,ghostface-killah,Hip-Hop,"Yo, I've seen Raj He been a friend for years N..."
294621,294621,yaar-hai-tu-pyaar-hai-tu,2012,atif-aslam,Rock,yaar hai tu pyaar hai tu justaju tu aarzoo tu ...
45302,45302,the-long-highway,2006,dire-straits,Rock,Heard a bell being toll Before the break of da...
24448,24448,clean,2015,big-data,Electronic,"When I look back, I see trails of myself. Coul..."
185008,185008,lady-sweet,2008,big-star,Rock,"Lady Sweet, I declare That you're the troubled..."
14244,14244,the-last-scene-of-struggling,2006,finger-eleven,Rock,The static grows and kills the message Unclear...


In [80]:
training_genre_lyrics = training_50k.filter([ 'genre', 'lyrics'])
validation_genre_lyrics = validation_50k.filter(['genre', 'lyrics'])
testing_genre_lyrics = testing_50k.filter(['genre', 'lyrics'])

In [81]:
testing_genre_lyrics.head()

,genre,lyrics
158198,Rock,"I am yours. However distant you may be, There ..."
29626,Country,MUST YOU THROW DIRT IN MY FACE (Bill Anderson)...
162772,Hip-Hop,Intro: When the bartender fuck your drink up I...
89746,Rock,It's Real Life (It's Real Life) It's Real Life...
320003,Rock,Second Street and Broadway Sittin' in a doorwa...


In [82]:
len(genre_lyrics_sample)

50000

In [83]:
genre_lyrics_sample['genre'].value_counts()

Rock          22495
Pop            8359
Metal          5019
Hip-Hop        4921
Country        3019
Jazz           1735
Electronic     1616
Other          1037
R&B             702
Indie           650
Folk            447
Name: genre, dtype: int64